## 트위터 토픽 모델링

In [ ]:
# 코랩 환경일 경우 실행
'''
from google.colab import drive
drive.mount('/content/drive')
'''

Mounted at /content/drive


In [ ]:
import pandas as pd
from pandas import DataFrame  as df
twitter = pd.read_csv("./data/twitter/트위터_환승연애.csv")
twitter

,date,tweet
0,2022-07-15 ~ 2022-07-16,환승연애2 x커플 모르겠고 이현나연 케미뢰전드.. \n심지어 저웅니들 둘이 룸메임
1,2022-07-15 ~ 2022-07-16,환승연애 최이현 여친짤 재질 미쳤음.. 아름다우셔
2,2022-07-15 ~ 2022-07-16,환승연애 첫선택 #menow
3,2022-07-15 ~ 2022-07-16,환승연애2 최이현 조승우 썰ㅋㅋ
4,2022-07-15 ~ 2022-07-16,환승연애2\n\n해가될까 \n\n#조승연 #WOODZ
...,...,...
5732,2022-11-10 ~ 2022-11-11,이진희도는 환승연애 나가면 안되겠다\n빽도깅이 불 지를지도 모름
5733,2022-11-10 ~ 2022-11-11,@onl_youu _youu 그 환승연애 과몰입에 저도 좀 끼워주세엽
5734,2022-11-10 ~ 2022-11-11,"(ENG) ""해은 제일 잘 나가"" 환승연애 보다가 냅다 케이팝 2시간 동안 갈긴 ..."
5735,2022-11-10 ~ 2022-11-11,다음 댓카에서 환승연애 토론 열어봅시다


In [ ]:
import numpy as np

#데이터 프레임의 'tweet' 열의 값들을 str 형식으로 바꾸기
twitter.tweet = twitter.tweet.astype(str)


#데이터 프레임의 'tweet' 열의 값 중 keyword1이나 keyword 2가 포함된 행만 Data에 저장
#clean_Data = Data.loc[Data['text'].str.contains('keyword1|keyword2')]

#데이터 프레임의 'text' 열의 값 중 keyword1이나 keyword 2가 포함된 행은 삭제
#clean_Data = Data[~Data['text'].str.contains('keyword1|keyword2')]

clean_Data = twitter

#text와 timestamp 열을 기준으로 중복된 데이터를 삭제, inplace : 데이터 프레임을 변경할지 선택(원본을 대체)
clean_Data.drop_duplicates(subset=['tweet'], inplace=True)

#한글이 아니면 빈 문자열로 바꾸기
clean_Data['tweet'] = clean_Data['tweet'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ',regex=True)

#빈 문자열 NAN 값으로 바꾸기
clean_Data = clean_Data.replace({'': np.nan})
clean_Data = clean_Data.replace(r'^\s*$', None, regex=True)

#NAN 이 있는 행은 삭제
clean_Data.dropna(how='any', inplace=True)

#인덱스 차곡차곡
clean_Data = clean_Data.reset_index (drop = True)

#데이터 프레임에 null 값이 있는지 확인
print(clean_Data.isnull().values.any()) 

False


In [ ]:
clean_Data

,date,tweet
0,2022-07-15 ~ 2022-07-16,환승연애 커플 모르겠고 이현나연 케미뢰전드 심지어 저웅니들 둘이 룸메임
1,2022-07-15 ~ 2022-07-16,환승연애 최이현 여친짤 재질 미쳤음 아름다우셔
2,2022-07-15 ~ 2022-07-16,환승연애 첫선택
3,2022-07-15 ~ 2022-07-16,환승연애 최이현 조승우 썰ㅋㅋ
4,2022-07-15 ~ 2022-07-16,환승연애 해가될까 조승연
...,...,...
4439,2022-11-10 ~ 2022-11-11,이진희도는 환승연애 나가면 안되겠다 빽도깅이 불 지를지도 모름
4440,2022-11-10 ~ 2022-11-11,그 환승연애 과몰입에 저도 좀 끼워주세엽
4441,2022-11-10 ~ 2022-11-11,해은 제일 잘 나가 환승연애 보다가 냅다 케이팝 시간 동안 갈긴 ...
4442,2022-11-10 ~ 2022-11-11,다음 댓카에서 환승연애 토론 열어봅시다


In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 353 kB/s 
     |████████████████████████████████| 465 kB 47.5 MB/s 


In [ ]:
from konlpy.tag import Komoran
komoran = Komoran()

komoran = Komoran(userdic='./data/단어 추가.txt')

#텍스트 데이터를 리스트로 변환
Data_list=clean_Data.tweet.values.tolist()

#불용어 제거
stopwords = ['거','것','은','는','이','가','때','문','뭐','수','말','씨','개','중','안','남','이랑', '그것', '이것', '화','번','건','애']


# 명사만
data_word=[]
for i in range(len(Data_list)):
    try:
        tagged = komoran.pos(Data_list[i])
        nouns = [s for s, t in tagged if t in ['NNP', 'NNG', 'NNB', 'NP'] and s not in stopwords]
        data_word.append(nouns)
    except Exception as e:
        continue

#트윗에서 명사만 추출해서 만든 리스트
data_word

In [ ]:
# 필요한 패키지를 import

from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


In [ ]:
# 고유한 단어들의 사전 만들기
dictionary = corpora.Dictionary(data_word)

In [ ]:
# 빈도가 2 이상인 단어만 + 전체의 60% 이상 차지하는 단어는 필터링(너무 등장하지 않거나, 너무 자주 등장하는 단어 필터링)
dictionary.filter_extremes(no_below=2, no_above=0.6)

In [ ]:
# 사전 속의 단어가 문장에서 몇 번 출현하는지 빈도를 세서 벡터화
corpus = [dictionary.doc2bow(text) for text in data_word]

In [ ]:
num_topics = 2 #생성될 토픽의 개수
chunksize = 2000 #한번의 트레이닝에 처리될 문서의 개수
passes = 10 #전체 코퍼스 트레이닝 횟수
iterations = 500 #문서 당 반복 횟수
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [ ]:
# 토픽이 어떻게 형성되었는지, Coherence는 적당하게 나왔는지 확인. 하나의 토픽 당 20개의 단어가 기본으로 보여짐

top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -4.5827.
[([(0.018584838, '나'),
   (0.011897345, '해은'),
   (0.011116749, '내'),
   (0.008210825, '얘기'),
   (0.0078645265, '환연'),
   (0.0072710207, '현규'),
   (0.0067390525, '사람'),
   (0.0066709984, '몰입'),
   (0.006403783, '생각'),
   (0.00637372, '연애'),
   (0.006179588, '희두'),
   (0.006154651, '티빙'),
   (0.0059113903, '나연'),
   (0.005856721, '커플'),
   (0.0055825408, '분'),
   (0.0055240546, '일'),
   (0.005426747, '편'),
   (0.005274886, '규민'),
   (0.005121813, '친구'),
   (0.004820846, '시간')],
  -4.338603218647293),
 ([(0.020057779, '연애'),
   (0.018392326, '나'),
   (0.016542034, '사람'),
   (0.016096931, '뱀뱀'),
   (0.010063547, '내'),
   (0.008008209, '환승'),
   (0.007588242, '년'),
   (0.0057974993, '데이트'),
   (0.0057337405, '환연'),
   (0.005629692, '듯'),
   (0.005544878, '남자'),
   (0.005205968, '프로그램'),
   (0.005157912, '생각'),
   (0.0051406585, '사실'),
   (0.0048790257, '엑스'),
   (0.0046371394, '질문'),
   (0.004554153, '선택'),
   (0.0044059367, '당신'),
   (0.0043480285, '마음'),

In [ ]:
!pip install pyLDAvis

In [ ]:
# 시각화 패키지 import

import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


In [ ]:
# html 파일로 시각화 자료 만들어짐

lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, './project_html/트위터토픽모델링.html')

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
